### Hyperparameters

In [212]:
# Hyperparameters

seqlen = 128
batch_size = 128
sample_size = 5

### Code

In [1]:
import keras as K
import os.path

import sys, imp

sys.path.append("../src")
from artstat import util
import numpy as np

Using TensorFlow backend.


In [3]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"

In [4]:
words, vocab = util.load_vocab("../vocab.txt", 10000)

In [5]:
emb_matrix = util.load_embeddings(vocab, 300, glove)

In [221]:
imp.reload(util)
X, Xu = util.load_data(path_train, vocab, pad=seqlen, numfiles=30)

  0%|          | 0/22860 [00:00<?, ?it/s]


In [25]:
from keras.layers import Reshape, Embedding, CuDNNLSTM, BatchNormalization, Dense, Concatenate
from keras import Model, Input
import tensorflow as tf

In [236]:
a = np.arange(5)
b = np.ones_like(a)

z = np.concatenate((np.expand_dims(b, axis=-1),
                np.expand_dims(a, axis=-1)),-1)
z.shape

(5, 2)

In [237]:
A = np.zeros((3, 10, 5))

for i in range(3):
    A[i,:,:] = np.reshape(np.array(range(50)), (10,5)) + i*100

#print(A)
    #A
#var = K.variable

indices = np.array([
    [[0,0], [0,5]],
    [[1,1], [1,6]],
    [[2,2], [2,7]]
], dtype="int32")
print(indices.shape)

#indices = np.array([0,5
#], dtype="int32")

v = K.backend.variable(A)
k = K.backend.variable(indices, dtype="int32")

pp = tf.shape(v)[0]
print("pp",pp)

g = tf.gather_nd(A, indices)

q = K.backend.eval(g)
print(q.shape)
print(q)

(3, 2, 2)
pp Tensor("strided_slice_2:0", shape=(), dtype=int32)
(3, 2, 5)
[[[  0.   1.   2.   3.   4.]
  [ 25.  26.  27.  28.  29.]]

 [[105. 106. 107. 108. 109.]
  [130. 131. 132. 133. 134.]]

 [[210. 211. 212. 213. 214.]
  [235. 236. 237. 238. 239.]]]


In [88]:
q = np.array([
    [0,1,2,0],
    [1,2,3,1],
    [2,3,4,2]
])

qv = K.backend.variable(q, dtype="int32")

def mygather(e):
    batch, indices = e
    print(type(batch))
    print(batch.shape)
    print(indices.shape)
    #i2 = tf.expand_dims(indices)
    #print(i2.shape)
    return (tf.gather(batch, indices),0)

r = tf.map_fn(mygather, (v, qv))

print(K.backend.eval(r[0]))

<class 'tensorflow.python.framework.ops.Tensor'>
(10, 5)
(4,)
[[[  0.   1.   2.   3.   4.]
  [  5.   6.   7.   8.   9.]
  [ 10.  11.  12.  13.  14.]
  [  0.   1.   2.   3.   4.]]

 [[105. 106. 107. 108. 109.]
  [110. 111. 112. 113. 114.]
  [115. 116. 117. 118. 119.]
  [105. 106. 107. 108. 109.]]

 [[210. 211. 212. 213. 214.]
  [215. 216. 217. 218. 219.]
  [220. 221. 222. 223. 224.]
  [210. 211. 212. 213. 214.]]]


In [243]:
def sampling_layer(x):
    data, sample_indices = x
    def gather_per_batch(e):
        batch, indices = e
        indices = tf.cast(indices, tf.int32)
        #print("batch", type(batch), batch.dtype, batch.shape)
        #print("indices", type(indices), indices.dtype, indices.shape)
        return (tf.gather(batch, indices),0)
    return tf.map_fn(gather_per_batch, (data, sample_indices),
                    parallel_iterations=batch_size*2, swap_memory=False)[0]


def sampling_layer_gather_nd(x):
    data, sample_indices = x
    return tf.gather_nd(data, tf.cast(sample_indices, tf.int32))


In [198]:
z = sampling_layer([v, qv])

K.backend.eval(z)

array([[[  0.,   1.,   2.,   3.,   4.],
        [  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.],
        [  0.,   1.,   2.,   3.,   4.]],

       [[105., 106., 107., 108., 109.],
        [110., 111., 112., 113., 114.],
        [115., 116., 117., 118., 119.],
        [105., 106., 107., 108., 109.]],

       [[210., 211., 212., 213., 214.],
        [215., 216., 217., 218., 219.],
        [220., 221., 222., 223., 224.],
        [210., 211., 212., 213., 214.]]], dtype=float32)

In [244]:
from keras.layers import Reshape, Embedding, CuDNNLSTM, BatchNormalization, Dense, \
    Concatenate, Lambda, Activation
from keras import Model, Input

def make_model(*, seqlen, sample_size, emb_matrix):    
    dim = emb_matrix.shape[1] + 1
    
    input_x = Input((seqlen,), dtype="int32", name="input_x")
    input_xu = Input((seqlen,), dtype="float32", name="input_xu")
    input_sample_indices = Input((sample_size,2), dtype="int32", name="input_sample_indices")

    resh_xu  = Reshape((seqlen,1), name="resh_xu")(input_xu)

    emb_layer = Embedding(*emb_matrix.shape, input_length=seqlen, 
                          trainable=False, weights=[emb_matrix], 
                          name="embedding")
    emb_x = emb_layer(input_x)
    concat_x = Concatenate(name="concat_x")([emb_x, resh_xu])
        
    yhat = CuDNNLSTM(128, return_sequences=True, name='rnn1')(concat_x)
    yhat = BatchNormalization()(yhat)
    yhat = CuDNNLSTM(128, return_sequences=False, name='rnn2')(yhat)

    yhat = BatchNormalization()(yhat)
    yhat = Dense(300, activation="relu")(yhat)
    yhat = BatchNormalization()(yhat)
    # len(vocab)+2 is because the zeroth word is for padding and last word is for "unknown"
    yhat = Dense(len(vocab)+2, activation="linear")(yhat)
    
    
    #print(input_sample_indices.dtype, input_sample_indices.shape)
    out_train = Lambda(sampling_layer_gather_nd, name="sampling")([yhat, input_sample_indices])
    out_train = Activation('softmax')(out_train)
    
    out_predict = Activation('softmax')(yhat)
    
    model_train = Model([input_x, input_xu, input_sample_indices], [out_train])
    model_predict = Model([input_x, input_xu], [out_predict])
    
    return model_train, model_predict

In [245]:
mtrain, mpredict = make_model(seqlen=seqlen, sample_size=sample_size, emb_matrix=emb_matrix)

In [246]:

mtrain.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_x (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_xu (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 128, 300)     3000300     input_x[0][0]                    
__________________________________________________________________________________________________
resh_xu (Reshape)               (None, 128, 1)       0           input_xu[0][0]                   
__________________________________________________________________________________________________
concat_x (

In [262]:
opt = K.optimizers.Adam(lr=0.01)
mtrain.compile(opt, loss='categorical_crossentropy')

In [263]:
imp.reload(util)
train_seq = util.NegativeSamplingPermutedSequence(data_x=X, data_xu=Xu,
                                                 seqlen=seqlen, batch_size=batch_size,
                                                 sample_size=sample_size,
                                                 vocab_size=len(vocab)+1)

In [264]:
print(len(X), seqlen*batch_size)

21201 16384


In [265]:
import math


numbatches = X.shape[0] // batch_size
mtrain.fit_generator(train_seq, steps_per_epoch=numbatches, epochs=50000
                   
                    )


Epoch 1/50000
165/165 [==============================] - 13s 76ms/step - loss: 0.1139
Epoch 2/50000
 50/165 [========>.....................] - ETA: 6s - loss: 0.1319

KeyboardInterrupt: 

In [ ]:
mpredict.compile(opt, loss="categorical_crossentropy")

In [181]:
len(words)

10001

In [186]:
i = 2999
gen = X[i:i+seqlen].tolist()
genu = Xu[i:i+seqlen].tolist()

tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype="float32")
results = []

UNK_IDX = len(words)

iterations = 32
for j in range(iterations):
    tX[0,:] = np.array(gen[-seqlen:], "int32")
    tXu[0,:] = np.array(genu[-seqlen:], "float32")
    
    #print(tX)
    z = mpredict.predict([tX, tXu])
 #   scores=z[0]
    idx = UNK_IDX
    #print(scores)
    while idx == UNK_IDX:
        idx = np.random.choice(range(len(vocab)+2), p=scores)

    gen.append(idx)
    genu.append(0.0)
    #print("idx", idx, UNK_IDX)
    results.append(words[idx])
    sys.stdout.write(results[-1] + " ")
    sys.stdout.flush()
#    print("%s ", results[-1])

long tragic conceptual selection way . interdisciplinary result film large programme multi international three place new film recent long multi global tour historical selection new spirit new new artist additional narrow new 

1.3547268e-16 0.93638355


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'